# 자살 전처리

## 데이터 불러오기
- 데이터 목록
    - 일별 자살자 수 통계 : [mdis](https://mdis.kostat.go.kr/dwnlSvc/ofrSurvSearch.do?curMenuNo=UI_POR_P9240) 사망원인통계
    - 우울증 관련 통계 : [보건의료빅데이터개방시스템](http://opendata.hira.or.kr/op/opc/olapMfrnIntrsIlnsInfo.do)
    - 소비자물가지수 : [KOSIS](https://kosis.kr/statHtml/statHtml.do?orgId=301&tblId=DT_005Y006&vw_cd=MT_ZTITLE&list_id=301_30106_4_2_2&scrId=&seqNo=&lang_mode=ko&obj_var_id=&itm_id=&conn_path=MT_ZTITLE&path=%252FstatisticsList%252FstatisticsListIndex.do)
    - 가계부채 : [가계신용 동향](https://www.index.go.kr/potal/main/EachDtlPageDetail.do?idx_cd=1076&param=003)
    - kospi 지수, 환율 : [FinanceDataReader](https://github.com/FinanceData/FinanceDataReader) 라이브러리
    - 기상데이터 : [기상청 기상자료개방포털](https://data.kma.go.kr/climate/RankState/selectRankStatisticsDivisionList.do)

## 사망원인통계 전처리(사망코드 55)

In [2]:
import os
files = os.listdir('./자살(2011-2020)/자살자_데이터분석용')
files

['a2011.csv',
 'a2012.csv',
 'a2013.csv',
 'a2014.csv',
 'a2015.csv',
 'a2016.csv',
 'a2017.csv',
 'a2018.csv',
 'a2019.csv',
 'a2020.csv']

In [3]:
import pandas as pd

for i in files:
     globals()[i.split('.')[0]] = pd.read_csv(f'./자살(2011-2020)/자살자_데이터분석용/{i}', encoding= 'euckr')

In [4]:
def engeering(df):
    
    df = df.dropna()
    df = df.rename( columns = {'사망자 주소(시도)':'locate', '사망연월일':'date', '사망원인 56항목 분류' : 'value'} )
    df = df[ df.value == 55 ] ## 사망원인 : 고의적 자해(자살)
    df['value'] = [1] * len(df)
    df = df.sort_values('date', ascending = True)
    df = df.reset_index(drop=True)
    df = df.groupby(['date', 'locate']).sum().reset_index()
    df['date'] = df['date'].astype(str).apply(pd.to_datetime).astype(str)
    
    return df

In [5]:
engeering(a2011)

,date,locate,value
0,2011-01-01,11,6
1,2011-01-01,21,2
2,2011-01-01,22,1
3,2011-01-01,23,2
4,2011-01-01,24,1
...,...,...,...
4718,2011-12-31,34,4
4719,2011-12-31,36,1
4720,2011-12-31,37,3
4721,2011-12-31,38,2


In [6]:
# 2011~2013, 2014~2020 feature 이름이 다르므로 각각 feature engeering 진행.

df_con = pd.concat([a2011, a2012, a2013])
df_con2 = pd.concat([a2014, a2015, a2016, a2017, a2018, a2019, a2020])

In [7]:
df_con = engeering(df_con)
df_con

,date,locate,value
0,2011-01-01,11,6
1,2011-01-01,21,2
2,2011-01-01,22,1
3,2011-01-01,23,2
4,2011-01-01,24,1
...,...,...,...
14005,2013-12-31,31,6
14006,2013-12-31,33,1
14007,2013-12-31,34,2
14008,2013-12-31,37,2


In [8]:
def engeering2(df):
    
    df = df.dropna()
    df = df.rename( columns = {'사망연월일':'date', '사망자주소행정구역시도코드':'locate', '사망원인_56항목분류코드' : 'value'} )
    df = df[ df.value == 55 ]
    df['value'] = [1] * len(df)
    df = df.sort_values('date', ascending = True)
    df = df.reset_index(drop=True)
    df = df.groupby(['date', 'locate']).sum().reset_index()
    df['date'] = df['date'].astype(str).apply(pd.to_datetime).astype(str)
    
    return df

In [9]:
df_con2.columns

Index(['사망연월일', '사망자주소행정구역시도코드', '사망원인_56항목분류코드'], dtype='object')

In [10]:
df_con2 = engeering2(df_con2)
df_con2

,date,locate,value
0,2014-01-01,11,6
1,2014-01-01,21,1
2,2014-01-01,22,1
3,2014-01-01,24,2
4,2014-01-01,26,1
...,...,...,...
32145,2020-12-31,31,6
32146,2020-12-31,33,1
32147,2020-12-31,34,3
32148,2020-12-31,37,2


In [11]:
df_sui = pd.concat([df_con, df_con2])

In [12]:
df_sui.shape

(46160, 3)

In [13]:
df_sui.to_excel('자살.xls', index = False)

<ipython-input-13-34daf563d886>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df_sui.to_excel('자살.xls', index = False)


In [14]:
import pandas as pd

df1 = pd.read_excel('자살.xls')
df1

,date,locate,value
0,2011-01-01,11,6
1,2011-01-01,21,2
2,2011-01-01,22,1
3,2011-01-01,23,2
4,2011-01-01,24,1
...,...,...,...
46155,2020-12-31,31,6
46156,2020-12-31,33,1
46157,2020-12-31,34,3
46158,2020-12-31,37,2


## 자살 데이터 지역코드(키값) 수정
- 기상청 데이터와 통계청 데이터의 지역코드가 서로 다르기 때문에 기상청 데이터에 맞게 지역코드를 수정한 feature 생성. 

- 통계청 지역코드
    - 11	서울특별시
    - 21	부산광역시
    - 22	대구광역시
    - 23	인천광역시
    - 24	광주광역시
    - 25	대전광역시
    - 26	울산광역시
    - 29	세종특별자치시
    - 31	경기도
    - 32	강원도
    - 33	충청북도
    - 34	충청남도
    - 35	전라북도
    - 36	전라남도
    - 37	경상북도
    - 38	경상남도
    - 39	제주특별자치도

- 기상청 지역코드
    - 11 서울경기
    - 32 강원영동
    - 33 충북
    - 34 충남
    - 35 전남
    - 36 전북
    - 37 경북
    - 38 경남
    - 39 제주

- 11, 23, 31 = 11로 통합
- 38, 21, 26 = 38
- 37, 22 = 37
- 36, 24 = 36
- 34, 25, 29 = 34

In [15]:
df1.rename(columns = {'locate':'loc_num'}, inplace = True)

In [16]:
df1['loc_numm'] = df1['loc_num']
df1

,date,loc_num,value,loc_numm
0,2011-01-01,11,6,11
1,2011-01-01,21,2,21
2,2011-01-01,22,1,22
3,2011-01-01,23,2,23
4,2011-01-01,24,1,24
...,...,...,...,...
46155,2020-12-31,31,6,31
46156,2020-12-31,33,1,33
46157,2020-12-31,34,3,34
46158,2020-12-31,37,2,37


In [17]:
df1.loc_numm = df1.loc_numm.replace(23, 11)
df1.loc_numm = df1.loc_numm.replace(31, 11)
df1.loc_numm = df1.loc_numm.replace(21, 38)
df1.loc_numm = df1.loc_numm.replace(26, 38)
df1.loc_numm = df1.loc_numm.replace(22, 37)
df1.loc_numm = df1.loc_numm.replace(24, 36)
df1.loc_numm = df1.loc_numm.replace(25, 34)
df1.loc_numm = df1.loc_numm.replace(29, 34)

In [18]:
df1.head()

,date,loc_num,value,loc_numm
0,2011-01-01,11,6,11
1,2011-01-01,21,2,38
2,2011-01-01,22,1,37
3,2011-01-01,23,2,11
4,2011-01-01,24,1,36


In [19]:
df1.loc_numm.value_counts()

11    10509
38     8696
37     6211
34     5965
36     5051
32     2814
35     2802
33     2706
39     1406
Name: loc_numm, dtype: int64

## 서브 데이터프레임 불러오기

- 파이썬으로 만든 kospi 등 금융 데이터와 기후 데이터 엑셀 종합본을 엑셀을 활용해 정리

In [20]:
df2 = pd.read_excel('종합.xls')

df2

,loc_num,date,loc_name,rain,temp,hum,sun,insola,exchange_rate,kospi,unemployment,price_index,house_debt,weekday,Unnamed: 14
0,32,2020-12-31,강원영동,0.0,-7.2,35,8.8,11.24,1084.47,2873.47,1135,100.330,1631.5,0,NaT
1,32,2020-12-30,강원영동,0.0,-7.4,35,6.8,9.89,1087.62,2873.47,1135,100.330,1631.5,0,NaT
2,32,2020-12-29,강원영동,0.0,3.1,63,1.7,5.34,1090.97,2820.51,1135,100.330,1631.5,0,NaT
3,32,2020-12-28,강원영동,0.0,5.7,60,7.9,9.46,1095.60,2808.60,1135,100.330,1631.5,0,NaT
4,32,2020-12-27,강원영동,0.0,5.5,50,4.1,7.83,1099.07,2806.86,1135,100.330,1631.5,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32872,33,2011-01-05,충북,0.1,-2.2,68,2.3,6.86,1123.50,2082.55,924,88.288,806.4,0,NaT
32873,33,2011-01-04,충북,0.0,-3.9,76,2.8,7.52,1125.50,2085.14,924,88.288,806.4,0,NaT
32874,33,2011-01-03,충북,0.5,-5.6,78,2.4,6.25,1124.65,2070.08,924,88.288,806.4,0,NaT
32875,33,2011-01-02,충북,0.0,-6.3,72,8.6,11.30,1124.65,2051.00,924,88.288,806.4,0,NaT


In [21]:
df2.drop('Unnamed: 14', axis = 1, inplace = True)
df2

,loc_num,date,loc_name,rain,temp,hum,sun,insola,exchange_rate,kospi,unemployment,price_index,house_debt,weekday
0,32,2020-12-31,강원영동,0.0,-7.2,35,8.8,11.24,1084.47,2873.47,1135,100.330,1631.5,0
1,32,2020-12-30,강원영동,0.0,-7.4,35,6.8,9.89,1087.62,2873.47,1135,100.330,1631.5,0
2,32,2020-12-29,강원영동,0.0,3.1,63,1.7,5.34,1090.97,2820.51,1135,100.330,1631.5,0
3,32,2020-12-28,강원영동,0.0,5.7,60,7.9,9.46,1095.60,2808.60,1135,100.330,1631.5,0
4,32,2020-12-27,강원영동,0.0,5.5,50,4.1,7.83,1099.07,2806.86,1135,100.330,1631.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32872,33,2011-01-05,충북,0.1,-2.2,68,2.3,6.86,1123.50,2082.55,924,88.288,806.4,0
32873,33,2011-01-04,충북,0.0,-3.9,76,2.8,7.52,1125.50,2085.14,924,88.288,806.4,0
32874,33,2011-01-03,충북,0.5,-5.6,78,2.4,6.25,1124.65,2070.08,924,88.288,806.4,0
32875,33,2011-01-02,충북,0.0,-6.3,72,8.6,11.30,1124.65,2051.00,924,88.288,806.4,0


In [22]:
df2.rename(columns = {'loc_num': 'loc_numm'},inplace = True)

In [23]:
df2

,loc_numm,date,loc_name,rain,temp,hum,sun,insola,exchange_rate,kospi,unemployment,price_index,house_debt,weekday
0,32,2020-12-31,강원영동,0.0,-7.2,35,8.8,11.24,1084.47,2873.47,1135,100.330,1631.5,0
1,32,2020-12-30,강원영동,0.0,-7.4,35,6.8,9.89,1087.62,2873.47,1135,100.330,1631.5,0
2,32,2020-12-29,강원영동,0.0,3.1,63,1.7,5.34,1090.97,2820.51,1135,100.330,1631.5,0
3,32,2020-12-28,강원영동,0.0,5.7,60,7.9,9.46,1095.60,2808.60,1135,100.330,1631.5,0
4,32,2020-12-27,강원영동,0.0,5.5,50,4.1,7.83,1099.07,2806.86,1135,100.330,1631.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32872,33,2011-01-05,충북,0.1,-2.2,68,2.3,6.86,1123.50,2082.55,924,88.288,806.4,0
32873,33,2011-01-04,충북,0.0,-3.9,76,2.8,7.52,1125.50,2085.14,924,88.288,806.4,0
32874,33,2011-01-03,충북,0.5,-5.6,78,2.4,6.25,1124.65,2070.08,924,88.288,806.4,0
32875,33,2011-01-02,충북,0.0,-6.3,72,8.6,11.30,1124.65,2051.00,924,88.288,806.4,0


## 지역코드 & date 조건으로 데이터 합치기

In [24]:
display(df1),
display(df2)

,date,loc_num,value,loc_numm
0,2011-01-01,11,6,11
1,2011-01-01,21,2,38
2,2011-01-01,22,1,37
3,2011-01-01,23,2,11
4,2011-01-01,24,1,36
...,...,...,...,...
46155,2020-12-31,31,6,11
46156,2020-12-31,33,1,33
46157,2020-12-31,34,3,34
46158,2020-12-31,37,2,37


,loc_numm,date,loc_name,rain,temp,hum,sun,insola,exchange_rate,kospi,unemployment,price_index,house_debt,weekday
0,32,2020-12-31,강원영동,0.0,-7.2,35,8.8,11.24,1084.47,2873.47,1135,100.330,1631.5,0
1,32,2020-12-30,강원영동,0.0,-7.4,35,6.8,9.89,1087.62,2873.47,1135,100.330,1631.5,0
2,32,2020-12-29,강원영동,0.0,3.1,63,1.7,5.34,1090.97,2820.51,1135,100.330,1631.5,0
3,32,2020-12-28,강원영동,0.0,5.7,60,7.9,9.46,1095.60,2808.60,1135,100.330,1631.5,0
4,32,2020-12-27,강원영동,0.0,5.5,50,4.1,7.83,1099.07,2806.86,1135,100.330,1631.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32872,33,2011-01-05,충북,0.1,-2.2,68,2.3,6.86,1123.50,2082.55,924,88.288,806.4,0
32873,33,2011-01-04,충북,0.0,-3.9,76,2.8,7.52,1125.50,2085.14,924,88.288,806.4,0
32874,33,2011-01-03,충북,0.5,-5.6,78,2.4,6.25,1124.65,2070.08,924,88.288,806.4,0
32875,33,2011-01-02,충북,0.0,-6.3,72,8.6,11.30,1124.65,2051.00,924,88.288,806.4,0


In [25]:
# date dtype datetime => str(object)

df2.date = df2.date.astype('str') 

In [26]:
df3 = pd.merge(df1, df2, how= 'left', on = ['loc_numm','date'])
df3

,date,loc_num,value,loc_numm,loc_name,rain,temp,hum,sun,insola,exchange_rate,kospi,unemployment,price_index,house_debt,weekday
0,2011-01-01,11,6,11,서울경기,0.0,-7.5,64,8.4,9.26,1124.65,2051.00,924,88.288,806.4,0
1,2011-01-01,21,2,38,경남,0.0,-2.0,46,6.1,9.85,1124.65,2051.00,924,88.288,806.4,0
2,2011-01-01,22,1,37,경북,0.9,-3.3,55,3.7,8.44,1124.65,2051.00,924,88.288,806.4,0
3,2011-01-01,23,2,11,서울경기,0.0,-7.5,64,8.4,9.26,1124.65,2051.00,924,88.288,806.4,0
4,2011-01-01,24,1,36,전남,0.4,-3.5,70,8.3,8.85,1124.65,2051.00,924,88.288,806.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46155,2020-12-31,31,6,11,서울경기,0.0,-8.7,57,6.3,8.68,1084.47,2873.47,1135,100.330,1631.5,0
46156,2020-12-31,33,1,33,충북,0.0,-9.4,63,6.1,11.02,1084.47,2873.47,1135,100.330,1631.5,0
46157,2020-12-31,34,3,34,충남,0.4,-7.6,75,6.5,9.67,1084.47,2873.47,1135,100.330,1631.5,0
46158,2020-12-31,37,2,37,경북,0.0,-6.6,48,8.7,11.22,1084.47,2873.47,1135,100.330,1631.5,0


In [27]:
df3.to_excel('자살통합본.xls', index = False)

<ipython-input-27-54eb2cfcc329>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df3.to_excel('자살통합본.xls', index = False)


## 우울증 통계 추가 및 분석 데이터 완성

**데이터를 일부 수정**
- 월별, 지역별로 구분된 우울증 로우 데이터를 엑셀 index, match 함수를 활용해 조건식을 만들어 컬럼 추가.
    - index(타깃범위,match(기준1,기준 범위1,0),match(기준2,기준 범위2,0))
- 같은 방식으로 기존 전국 월별 실업자 수 통계를 지역별로 세분화해 컬럼에 추가.
- 엑셀의 text 함수를 활용해 'weekday' 컬럼 추가.
    - TEXT(cell,"aaa")

In [28]:
import xlwings as xw
book = xw.Book('전처리최종.xls')

In [29]:
df = book.sheets(1).used_range.options(pd.DataFrame).value
df = df.reset_index()
df

,loc_num,date,year,month,day,weekday,loc_numm,loc_name,rain,temp,hum,sun,insola,exchange_rate,kospi,price_index,house_debt,patient,unemployment,suicide
0,11.0,2011-01-01,2011.0,1.0,1.0,토,11.0,서울경기,0.0,-7.5,64.0,8.4,9.26,1124.65,2051.00,88.288,806.4,NaN,252.0,6.0
1,21.0,2011-01-01,2011.0,1.0,1.0,토,38.0,경남,0.0,-2.0,46.0,6.1,9.85,1124.65,2051.00,88.288,806.4,NaN,64.0,2.0
2,22.0,2011-01-01,2011.0,1.0,1.0,토,37.0,경북,0.9,-3.3,55.0,3.7,8.44,1124.65,2051.00,88.288,806.4,NaN,51.0,1.0
3,23.0,2011-01-01,2011.0,1.0,1.0,토,11.0,서울경기,0.0,-7.5,64.0,8.4,9.26,1124.65,2051.00,88.288,806.4,NaN,76.0,2.0
4,24.0,2011-01-01,2011.0,1.0,1.0,토,36.0,전남,0.4,-3.5,70.0,8.3,8.85,1124.65,2051.00,88.288,806.4,NaN,22.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46155,31.0,2020-12-31,2020.0,12.0,31.0,목,11.0,서울경기,0.0,-8.7,57.0,6.3,8.68,1084.47,2873.47,100.330,1631.5,80473.0,274.0,6.0
46156,33.0,2020-12-31,2020.0,12.0,31.0,목,33.0,충북,0.0,-9.4,63.0,6.1,11.02,1084.47,2873.47,100.330,1631.5,11591.0,26.0,1.0
46157,34.0,2020-12-31,2020.0,12.0,31.0,목,34.0,충남,0.4,-7.6,75.0,6.5,9.67,1084.47,2873.47,100.330,1631.5,15193.0,40.0,3.0
46158,37.0,2020-12-31,2020.0,12.0,31.0,목,37.0,경북,0.0,-6.6,48.0,8.7,11.22,1084.47,2873.47,100.330,1631.5,14348.0,57.0,2.0


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46160 entries, 0 to 46159
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   loc_num        46160 non-null  float64
 1   date           46160 non-null  object 
 2   year           46160 non-null  float64
 3   month          46160 non-null  float64
 4   day            46160 non-null  float64
 5   weekday        46160 non-null  object 
 6   loc_numm       46160 non-null  float64
 7   loc_name       46160 non-null  object 
 8   rain           46160 non-null  float64
 9   temp           46160 non-null  float64
 10  hum            46160 non-null  float64
 11  sun            46160 non-null  float64
 12  insola         46160 non-null  float64
 13  exchange_rate  46160 non-null  float64
 14  kospi          46160 non-null  float64
 15  price_index    46160 non-null  float64
 16  house_debt     46160 non-null  float64
 17  patient        36403 non-null  float64
 18  unempl

In [31]:
df.to_csv('suicide.csv', index = False)